# Using `bash` in bioinformatics

The Unix commands shown in this notebook are quite intimidating at first. There's a lot of material becaue I try to cover all the stuff you willl need to construct a typicla bioinformatics pipeline. 

Try to understand the expressions and play aound to see what varying them will do. You do not need to memorize the commands now - you can always come back to this notebook for reference. You will also get more practice when you do the Bioiinformatics sessions to construct Unix pipelines for processing your RNA-seq data sets.

Hack to handle broken pipes.

In [ ]:
cleanup () { 
    :
}

trap "cleanup" SIGPIPE

Treat unset variables as an error, and immediately exit.

In [ ]:
set -u

## 0.1. Files, directories and working with text

Quickly reveiw what the following navigation and file/directory manipulation commands do:

```bash
>, >>, <, |, ls, cd, pwd, cp, mv, rm, mkdir, rmdir
```

This session dives quite deep into Unix shell scripting and invovles quite a bit of text processing. We review some basic commands for working with text in Unix.

#### This is the file we will warm up with

We create it using a HEREDOC that allows us to construct multi-line files and redirrect it to the file `junk.txt`

In [ ]:
cat > junk.txt <<EOF 
1. CCCTAACCCCCTAACCCCCTAACCCTCAGTCGGGGAGGCGACAATAGCTG
2. GTCATATGTTCTGTACGTTATTGGCCAACTGATCATACCTGAATCGAGCC
3. GAACCGGGATTATCAAAGACGAACATGGTCGGGTCCTTGAACCAAACGAA
4. TCTCCGTCCGCTGGCGTGTTTTTCTTTTCTCAAGTGGGCAAGTTACCCGG
EOF

#### We can see the contents of a file with `cat`

In [ ]:
cat junk.txt

#### Using `head` and `tail`

In [ ]:
head -3 junk.txt

In [ ]:
tail -3 junk.txt

In [ ]:
tail +3 junk.txt

#### Pipling to chain commnads

In [ ]:
head -3 junk.txt | tail -2

#### Searching within a file with `grep`

- `grep` takes a REGEX (regular expression) as the query and a file (or files) to search
- Normally grep shows lines that match the REGEX argument
- The `-v` flag shows the lines that DO NOT match the REGEX argument

In [ ]:
grep "[2|4]" junk.txt

In [ ]:
grep -v "[2|4]" junk.txt

#### Use a regular expression with `grep`

In [ ]:
grep "CCC.*CCC" junk.txt

#### Get rid of numbers with `cut`

`cut` extracts columns separated by delimiters
- here the delimiter is a space `-d ' '`
- and we cut the second column `-f 2`

In [ ]:
cut -d ' ' -f 2 junk.txt

#### Get rid of numbers with `grep`

The `-o` flag to `grep` shows only the strings mathcing the regular expression

In [ ]:
grep -o '[A-Z]*' junk.txt

#### Convert DNA to RNA with `tr`

In [ ]:
cat junk.txt | tr T U

#### Reverse complement with `tr`

In [ ]:
grep -o '[ACTG]*' junk.txt |
tr ACGT TGCA |
rev

#### Split command over multiple lines

Use `\` to continue command to next line

In [ ]:
grep -o '[ACTG]*' junk.txt | tr \
ACGT TGCA | rev

Actually the line continuation character is not necesssary if you break at a pipe `|`

In [ ]:
grep -o '[ACTG]*' junk.txt | 
tr ACGT TGCA | 
rev

## 0.2 Getting files

We will download Chromosomes 7 and 8 of Crytococcus neoformans in FASTA format from NCBI Nucleotide. We can find their IDs through a [web query](https://www.ncbi.nlm.nih.gov/nuccore/?term=cryptococcus+neoformans+B-3501A).

We find that the IDs are

```
CM000046.1 for chormosome 7
CM000047.1 for chormosome 8
```

We will use the NCBI Entrez tool `efetch` to download the FASTA files to a folder `seqs`.

In [ ]:
cd
mkdir -p seqs
efetch -db=nuccore -format=fasta -id=CM000046.1 > seqs/CM000046.1.fa
efetch -db=nuccore -format=fasta -id=CM000047.1 > seqs/CM000047.1.fa

In [ ]:
ls seqs

In [ ]:
head seqs/CM000046.1.fa

In [ ]:
head seqs/CM000047.1.fa

## 0.3 Compressing ard archiving files

### Using `zip`, `gzip`, `bzip2`, `bgzip`

In Bioinformatics the most common formats are `gzip` and `bgzip`.

In [ ]:
cd ~/seqs
ls

#### Compressing with `gzip`

In [ ]:
gzip *fa
ls

#### Uncompressing with `gzip`

In [ ]:
gunzip *gz
ls 

#### Compressing with `bgzip`

`bgzip` (Blocked GNU Zip Format) is commonly used in Bioinformatics, and allows random access of the content. BAM, BCF and VCF file formats are typically `bgzip` compressed.

Unfortunately `bgzip` does not take multiple files, and we have to use the `xargs` command to pipe in one file at a time.

In [ ]:
xargs --help | grep '\-n,'

In [ ]:
ls *fa | xargs -n1 bgzip
ls

#### Note that `bgzip` files can be uncompressed with `gzip`

In [ ]:
gunzip *gz
ls

### Viewing compressed files

Some utilities can work directly with gzipped files. One that we will be using extensively is `zcat`.

In [ ]:
ls *fa | xargs -n1 bgzip
ls

In [ ]:
zcat CM000046.1.fa.gz | head

In [ ]:
gunzip *gz
ls

### Archival with `tar`

Often we want to compress an entire direcotry (possibly with subdirectories). In Unix, this is most commonly done with `tar`. Flags to note

- `c` compress
- `x` extract 
- `z` gzip or gunzip
- `f` files
- `v` verbose

In [ ]:
tar --help | head -8

In [ ]:
cd 
ls seqs

In [ ]:
tar czvf seqs.tar.gz seqs
ls seqs*

#### View contents of archive

In [ ]:
tar tzvf seqs.tar.gz

#### Remove direcotry and regenerate from archive

In [ ]:
rm -rf seqs
ls seqs*

In [ ]:
tar xzvf seqs.tar.gz
ls seqs*

In [ ]:
rm seqs.tar.gz

## 0.4 Checksums

When working with genomic data, we deal with very large files. There is a small risk that these files will be corrupted over time or during data transfer. To ensure that files are not changed, we use a “checksum” function. This is a function that generates an long, essentially random number called a checksum that represents the contents of the file. When the file contents change, so will the checksum. In theory, there is a very small probability that two different files generate the same checksum, but in practice the probability is too small to worry about.

There are several different algorithms for generating the checksums, and at least 3 Unix commands to do so, but they all work very similarly for our purposes.

The strategy is:

- Generate and store a checksum together with a data file whose integrity you care about
- When you use or download the data, re-generate the checksum (using the same algorithm e.g. MD5) and compare with the checksum

In [ ]:
ls ~/seqs

In [ ]:
md5sum ~/seqs/*fa > MD5_CHECKSUM

#### We check that the files are unchanged wth the `-c` flag

In [ ]:
md5sum -c MD5_CHECKSUM

#### Suppsoe one of the files is corrupted

In [ ]:
echo 'Oops I did it again' >> seqs/CM000047.1.fa

In [ ]:
md5sum -c MD5_CHECKSUM  \
| cat # this is jusst so we can Run All

#### Using `diff` to find changes

Let's rename the altered file, re-download the orginal and see what was changed.

In [ ]:
mv seqs/CM000047.1.fa seqs/CM000047.1.fa.bak

In [ ]:
efetch -db=nuccore -format=fasta -id=CM000047.1 > seqs/CM000047.1.fa

In [ ]:
diff -u seqs/CM000047.1.fa seqs/CM000047.1.fa.bak \
| cat # this is jusst so we can Run All

Explanaiton from [official documentation]

Header

```
--- seqs/CM000047.1.fa	2018-06-21 09:58:18.246185758 -0400
+++ seqs/CM000047.1.fa.bak	2018-06-21 09:55:55.967707764 -0400
```

Explanation

```
The unified output format starts with a two-line header, which looks like this:

--- from-file from-file-modification-time
+++ to-file to-file-modification-time
The timestamp looks like ‘2002-02-21 23:30:39.942229878 -0800’ to indicate the date, time with fractional seconds, and time zone. The fractional seconds are omitted on hosts that do not support fractional timestamps.

You can change the header’s content with the --label=label option. See Alternate Names.
```

Hunk 

```
@@ -16820,3 +16820,4 @@
 CCGTGTCGGCACGCATCTGATTTTTTTCTTATTATTCATAACAACGGCTACAGATCACGCAAGCGCATCG
 ACTCCGCTGCTGCGAAACACCACGAGTACTGCAACCATTGCTGCGGATGAGCTTAGGGGGTTAGGGGGTT
 AGGGGGTTAGGGGGTTAGGGGGTTAGGGGGTTAGGGGGTTAGGGGGTTAGGGG
+Oops I did it again
```

Explanation

```
@@ from-file-line-numbers to-file-line-numbers @@
 line-from-either-file
 line-from-either-file…
 
 If a hunk contains just one line, only its start line number appears. Otherwise its line numbers look like ‘start,count’. An empty hunk is considered to start at the line that follows the hunk.

If a hunk and its context contain two or more lines, its line numbers look like ‘start,count’. Otherwise only its end line number appears. An empty hunk is considered to end at the line that precedes the hunk.

The lines common to both files begin with a space character. The lines that actually differ between the two files have one of the following indicator characters in the left print column:

‘+’
A line was added here to the first file.

‘-’
A line was removed here from the first file.
 ```

#### Verify

We can confirm that a line was added at the position given by `diff`

In [ ]:
cat seqs/CM000047.1.fa | tail +16820 | head -3

In [ ]:
cat seqs/CM000047.1.fa.bak | tail +16820 | head -4

## 1. Working with FASTA files

### Strucrure of a FASTA file

From [NCBI FASTA specification](https://blast.ncbi.nlm.nih.gov/Blast.cgi?CMD=Web&PAGE_TYPE=BlastDocs&DOC_TYPE=BlastHelp)

```
A sequence in FASTA format begins with a single-line description, followed by lines of sequence data. The description line (defline) is distinguished from the sequence data by a greater-than (">") symbol at the beginning. It is recommended that all lines of text be shorter than 80 characters in length. An example sequence in FASTA format is:

		>P01013 GENE X PROTEIN (OVALBUMIN-RELATED)
		QIKDLLVSSSTDLDTTLVLVNAIYFKGMWKTAFNAEDTREMPFHVTKQESKPVQMMCMNNSFNVATLPAE
		KMKILELPFASGDLSMLVLLPDEVSDLERIEKTINFEKLTEWTNPNTMEKRRVKVYLPQMKIEEKYNLTS
		VLMALGMTDLFIPSANLTGISSAESLKISQAVHGAFMELSEDGIEMAGSTGVIEDIKHSPESEQFRADHP
		FLFLIKHNPTNTIVYFGRYWSP
```

Note that a FASTA file may contain mulitple sequences, each with its own description line.

#### Set up variables for files we will be using

In [ ]:
DATA_DIR='/data/hts/2018/Granek_4837_180427A5'
FASTA='cneo040623.scaffs.fa.gz'
FASTQ='10_MA_C_S3_L003_R1_001.fastq.gz'
GTF='Cryptococcus_neoformans.ASM9104v1.39.gtf.gz'

#### This is how you get the value of a shell variable

In [ ]:
echo $DATA_DIR

#### List files in data direcotry

In [ ]:
ls $DATA_DIR | head -3

#### Inspect the contents of a zipped file.

`zcat` is like `cat` but works on gzipped files.

In [ ]:
zcat $DATA_DIR/$FASTA | head

#### Show headers only

In [ ]:
zcat $DATA_DIR/$FASTA   | grep '^>' | head

#### Show headers with line numbers

In [ ]:
zcat $DATA_DIR/$FASTA| grep -n '^>' | head

#### Count number of headers (one for each crhomosome)

In [ ]:
zcat $DATA_DIR/$FASTA  | grep -c '^>'

#### Extract first part of the sequence from chromosome 7

In [ ]:
zcat $DATA_DIR/$FASTA | sed -n '210079,210082p'

#### Extract first line of chromosome 7 and save to file

In [ ]:
zcat $DATA_DIR/$FASTA | sed -n '210080p' > dna.txt

#### Inspect the file we just saved

In [ ]:
cat dna.txt

#### Find complement

In [ ]:
cat dna.txt | tr ACGT TGCA

#### Find reverse complement

In [ ]:
cat dna.txt | tr ACGT TGCA | rev

#### Delete A and T

In [ ]:
cat dna.txt | tr -d 'AT' 

### Only count non A and T

In [ ]:
cat dna.txt | tr -d 'AT' | tr -d '\n' | wc -c

In [ ]:
man tr | grep '\-d,' -A2

#### Count C and G

In [ ]:
grep '[CG]' -o dna.txt | wc -l

In [ ]:
man grep | grep '\-o,' -A2

#### Specify data directory as a shell variable

## 2. Working with FASTQ files

### Structure of a FASTQ file

From [Wikipedia](https://en.wikipedia.org/wiki/FASTQ_format)

```
A FASTQ file normally uses four lines per sequence.

Line 1 begins with a '@' character and is followed by a sequence identifier and an optional description (like a FASTA title line).
Line 2 is the raw sequence letters.
Line 3 begins with a '+' character and is optionally followed by the same sequence identifier (and any description) again.
Line 4 encodes the quality values for the sequence in Line 2, and must contain the same number of symbols as letters in the sequence.
```

#### Specify target file as a shell variable

#### Show the contents of a zipped file

In [ ]:
zcat $DATA_DIR/$FASTQ | head -n 10 

#### Extracting just the sequence

We need to extract every 4th line starting from the 2nd.  We cna use a tool called `sed` (stream editor) to do so.

In [ ]:
zcat $DATA_DIR/$FASTQ | sed -n '2~4p;' | head

**Explanation**

`sed -n '2~4p'`

- `-n` means to suppress all output
- `2~4p` means print (p) every 4th line starting at line 2

#### Joining lines

Getting sequence as a single string. We just need to delete newlline (\n) characters.

Note: We can break long commands uisng the line continuation character \

In [ ]:
zcat $DATA_DIR/$FASTQ | \
sed -n '2~4p;' | \
head | \
tr -d '\n' 

## 3.Working wtih GTF files

### Structure of a GTF file

From [UCSD Genome Browser](http://genome.ucsc.edu/FAQ/FAQformat.html#format3)

```
The GTF (General Transfer Format) is an extension of GFF version 2 
and used to represent transcription models. GFF (General Feature Format) 
consists of one line per feature, each containing 9 columns of data. 

Fields

Fields are tab-separated. Also, all but the final field in each 
feature line must contain a value; "empty" columns are denoted 
with a '.'

    seqname   - name of the chromosome or scaffold; chromosome names 
                without a 'chr' 
    source    - name of the program that generated this feature, or 
                the data source (database or project name)
    feature   - feature type name. Current allowed features are
                {gene, transcript, exon, CDS, Selenocysteine, start_codon,
                stop_codon and UTR}
    start     - start position of the feature, with sequence numbering 
                starting at 1.
    end       - end position of the feature, with sequence numbering 
                starting at 1.
    score     - a floating point value indiciating the score of a feature
    strand    - defined as + (forward) or - (reverse).
    frame     - one of '0', '1' or '2'. Frame indicates the number of base pairs
                before you encounter a full codon. '0' indicates the feature 
                begins with a whole codon. '1' indicates there is an extra
                base (the 3rd base of the prior codon) at the start of this feature.
                '2' indicates there are two extra bases (2nd and 3rd base of the 
                prior exon) before the first codon. All values are given with
                relation to the 5' end.
    attribute - a semicolon-separated list of tag-value pairs (separated by a space), 
                providing additional information about each feature. A key can be
                repeated multiple times.

Attributes

The following attributes are available. All attributes are semi-colon
separated pairs of keys and values.

- gene_id: The stable identifier for the gene
- gene_version: The stable identifier version for the gene
- gene_name: The official symbol of this gene
- gene_source: The annotation source for this gene
- gene_biotype: The biotype of this gene
- transcript_id: The stable identifier for this transcript
- transcript_version: The stable identifier version for this transcript
- transcript_name: The symbold for this transcript derived from the gene name
- transcript_source: The annotation source for this transcript
- transcript_biotype: The biotype for this transcript
- exon_id: The stable identifier for this exon
- exon_version: The stable identifier version for this exon
- exon_number: Position of this exon in the transcript
- ccds_id: CCDS identifier linked to this transcript
- protein_id: Stable identifier for this transcript's protein
- protein_version: Stable identifier version for this transcript's protein
- tag: A collection of additional key value tags
- transcript_support_level: Ranking to assess how well a transcript is supported (from 1 to 5)
```

#### Inspect the GTF file

In [ ]:
zcat $DATA_DIR/$GTF | head

#### Ignore comment lines version 1

This uses `tail` to take all lines starting from line 6

In [ ]:
zcat $DATA_DIR/$GTF | tail +6 | head -3

#### Ingore comment lines 2

This uses `grep` to ignore (`-v`) lines starting with `^#`

In [ ]:
zcat $DATA_DIR/$GTF | grep -v '^#' | head -3

#### Find all unique feature types in GTF

In [ ]:
zcat $DATA_DIR/$GTF | grep -v '^#' | cut -f 3 | sort | uniq

#### Get counts of all unique feature types in GTF

In [ ]:
zcat $DATA_DIR/$GTF | grep -v '^#' | cut -f 3 | sort | uniq -c

#### Find the the attributes of the first 3 exons

In [ ]:
zcat $DATA_DIR/$GTF | grep -v '^#' | awk '$3 == "exon"' | head -3

#### Store attributes of all genes in a file

It is common to use Unix commands to do simple processing and save the results to a file, then use a scripting language such as R or Python to furhter process the file.

In [ ]:
zcat $DATA_DIR/$GTF | grep -v '^#' | awk '$3 == "gene"' | cut -f 9 > genes.txt

In [ ]:
head -6 genes.txt

## 4. Looping

#### Loop throgh first 10 lines

- Creae a counter variable `LINE_NUM` and initialize it to 0
- Redirect `genes.txt` to a while loop
- Whlie loop reads each line into the variable `LINE`
- The `[[ CONDITION ]]` tests if CONDITION is true
- If LINE_NUM is less than `-lt` 10 print LINE and increment LINE_NUM
- Note that `(( EXPR ))` is used to evaluate numeric expressions

In [ ]:
LINE_NUM=0
while read LINE
do
    if [[ $LINE_NUM -lt 10 ]]; then
        echo $LINE
        (( LINE_NUM++ ))
    fi
done < genes.txt

#### Loop through first 10 tRNAs

Almost the same as previous program but we add a new condition to check tha the gene_biotype is tRNA using a REGULAr EXPRESSION via the match operator `=~`

`.*gene_biotype[[:space:]]\"tRNA\".*` means

- `.*`: Match any number of any character 
- `gene_biotype`: Match the string gene_biotype
- `[[:space:]]`: Match a single space cahracter
- `\"tRNA\"`: Math the string "tRNA" - we escape the double quotes to match linteral double quote characters
- `.*`: Match any number of any character 

In [ ]:
LINE_NUM=0
while read LINE
do
    if [[ $LINE_NUM -lt 10 ]] && [[ $LINE =~ .*gene_biotype[[:space:]]\"tRNA\".* ]] ; then
        echo $LINE
        ((LINE_NUM++))
    fi
done < genes.txt

#### Loop through files in a directory and print the filename followed by the first line

- Use a COMMAND EXPRESSION `$(EXPR)` to list files matching the glob pattern
- We loop through each file using a `for` statement
- For each file, extract the first line with `head -1` and save in variable `FIRST`
- The `2>/dev/null` ignores output from the `stderr` stream
- We use another COMMAND EXPRESSION to print the filename withou the directory (using `basename`)
- We then print the first line storred in `FIRST`

In [ ]:
for FILE in $(ls $DATA_DIR/*_MA_J_S20_*gz)
do
    FIRST=$(zcat $FILE 2>/dev/null | head -1)
    echo $(basename $FILE)
    echo $FIRST
    echo
done